In [33]:
import os
import re
import requests
import numpy as np

from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from urllib.parse import urlparse

from tqdm import tqdm


In [49]:
class URLCrawler():
    def __init__(self, categories) -> None:
        if type(categories) !=  list:
            self.categories = [categories]
            print(self.categories)
        else:
            self.categories = categories
        self.path = './Data/'
        self.makeDirectory()

 
    def checkURL(self, requested_url):
        if not urlparse(requested_url).scheme:
            requested_url = "https://" + requested_url
        return requested_url


    def requestAndParse(self, requested_url):
        requested_url = self.checkURL(requested_url)
        try:
            # define headers to be provided for request authentication
            headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
                            'AppleWebKit/537.11 (KHTML, like Gecko) '
                            'Chrome/23.0.1271.64 Safari/537.11',
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
                'Accept-Encoding': 'none',
                'Accept-Language': 'en-US,en;q=0.8',
                'Connection': 'keep-alive'}
            request_obj = Request(url=requested_url, headers=headers)
            page_html = urlopen(request_obj)
            page_soup = BeautifulSoup(page_html, "html.parser")
            return page_soup

        except Exception as e:
            print(e)

    def crawlImageURL(self, soupObject):
        collection = []
        soup = soupObject
        for img in soup.find_all('img'):
            collection.append(img.attrs['src'])
        collection = set(collection) #Use set to remove duplicate values.
        return list(collection)

    def makeDirectory(self):
        for name in self.categories:
            path = os.path.join(self.path, name)
            if not os.path.exists(path): 
                os.mkdir(path)
                print('{} IS CREATED!!!'.format(name))
            else:
                print('{} IS EXIST!!!'.format(path))

    def downloadImgWithURL(self, urlCollection, category):

        for url in urlCollection:
            indexes = re.split('/|\?', url)
            for idx in indexes:
                if re.search('\d\.(jpg|png|jpeg)', idx):
                    filename = idx
                else:
                    filename = category + '_' + str(hash(np.random.randint(0, 100000))) + '_' + str(hash(np.random.randint(0, 100000))) + '.jpg'
                    # filename = idx[-2] + '.jpg'
                    
            response = requests.get(url, stream=True)
            if response.status_code == 200:
                response.raw.decode_content = True
                open(self.path + category + '/' + filename, 'wb').write(response.content)

    def download(self, website):
        urlList = []
        if type(website) != list:
            urlList.append(website)
        else:
            urlList = website

        for web in website:
            print('CRAWILING AT ' + web)
            for category in self.categories:
                print('CRAWLING IMAGE OF ' + category.upper())
                soupObject = self.requestAndParse(web + category + '/')
                imgCollection = self.crawlImageURL(soupObject)
                self.downloadImgWithURL(imgCollection, category)



In [129]:


# soupObject = requestAndParse('https://www.freeimages.com/search/cat')
# imgCollection = crawlImageURL(soupObject)
# imgCollection

# for animal in tqdm(animal_list):
#     soupObject = requestAndParse(url_resources[0] + animal + '/')
#     imgCollection = crawlImageURL(soupObject)
#     downloadImgWithURL(imgCollection, animal)


100%|██████████| 11/11 [06:36<00:00, 36.06s/it]


In [50]:
url_resources = ['https://www.pexels.com/search/', 'https://www.freeimages.com/search/', 'https://unsplash.com/s/photos/']

animal_list = ['cat', 'lion', 'leopard', 'tiger', 'jaguar', 'sphynx', 
                'dog', 'wolf', 'husky', 'corgi', 'pug']

crawling = URLCrawler(animal_list)
crawling.download(url_resources)

cat IS CREATED!!!
lion IS CREATED!!!
leopard IS CREATED!!!
tiger IS CREATED!!!
jaguar IS CREATED!!!
sphynx IS CREATED!!!
dog IS CREATED!!!
wolf IS CREATED!!!
husky IS CREATED!!!
corgi IS CREATED!!!
pug IS CREATED!!!
DOWNLOADING IMAGE OF CAT


  0%|          | 0/3 [00:00<?, ?it/s]

CRAWLING AT https://www.pexels.com/search/


 33%|███▎      | 1/3 [00:10<00:20, 10.21s/it]

CRAWLING AT https://www.freeimages.com/search/


 33%|███▎      | 1/3 [00:10<00:21, 10.90s/it]

HTTP Error 410: Gone


AttributeError: 'NoneType' object has no attribute 'find_all'

In [34]:
# soupObject = crawling.requestAndParse('cat')
# imgCollection = crawling.crawlImageURL(soupObject)
# crawling.downloadImgWithURL(imgCollection, 'cat')

# imgCollection
'CAT_' + str(hash(np.random.randint(0, 10000)))

'CAT_1096'